In [ ]:
# 1. Crear la SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("Titanic MLib Activity")\
    .getOrCreate()

spark

## 2. Carga del dataset Titanic

En esta sección cargaremos el archivo `titanic.csv` ubicado en la carpeta de la sesión:

- Ruta esperada (en este entorno): `M8/S4/titanic.csv`.
- Asegúrese de que la ruta relativa sea correcta si mueve el notebook.

También inspeccionaremos el esquema y algunas filas para entender la estructura de los datos.

In [ ]:
# 2. Cargar el archivo titanic.csv
import os

data_path = os.path.join("titanic.csv")  # Ajustar si es necesario

df = spark.read.csv(data_path, header=True, inferSchema=True)

print(f"Número de filas: {df.count()}")
df.printSchema()
df.show(5, truncate=False)

## 3. Análisis exploratorio básico

Realice un análisis exploratorio para entender mejor los datos:
- Distribución de la variable objetivo (por ejemplo, `Survived`).
- Revisión de valores nulos.
- Variables categóricas vs numéricas.

A modo de ejemplo, a continuación se incluye un análisis mínimo. Puede ampliarlo según lo requiera el encargo.

In [ ]:
# 3.1 Distribución de la variable objetivo
from pyspark.sql import functions as F

target_col = 'Survived'  # Ajustar si el dataset usa otro nombre

df.groupBy(target_col).count().orderBy(target_col).show()

# 3.2 Conteo de valores nulos por columna
null_counts = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns])
null_counts.show(truncate=False)

## 4. Preprocesamiento y construcción del Pipeline

En esta sección construiremos un **Pipeline de Spark ML** que incluya:

1. Transformadores de preprocesamiento:
   - `StringIndexer` para variables categóricas.
   - `OneHotEncoder` para generar variables dummy.
   - `VectorAssembler` para combinar todas las features en un solo vector.
2. Un estimador de clasificación (por ejemplo, `RandomForestClassifier`).
3. Un proceso de validación y búsqueda de hiperparámetros (ParamGridBuilder + CrossValidator).

Primero seleccionaremos las columnas que vamos a usar como features y definiremos cuáles son categóricas y cuáles numéricas.

In [ ]:
# 4.1 Definir columnas de features y variable objetivo
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

label_col = 'Survived'  # variable objetivo

# Ejemplo de selección de columnas (ajustar según el esquema real del CSV)
numeric_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
categorical_cols = ['Sex', 'Embarked']

# Indexación de la etiqueta (por si no está en formato numérico)
label_indexer = StringIndexer(inputCol=label_col, outputCol='label', handleInvalid='keep')

# Indexadores para variables categóricas
indexers = [
    StringIndexer(inputCol=c, outputCol=f'{c}_idx', handleInvalid='keep')
    for c in categorical_cols
]

# OneHotEncoder para variables categóricas indexadas
encoder = OneHotEncoder(
    inputCols=[f'{c}_idx' for c in categorical_cols],
    outputCols=[f'{c}_ohe' for c in categorical_cols],
)

# Columnas finales para el ensamblador
assembler_inputs = numeric_cols + [f'{c}_ohe' for c in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol='features')

assembler_inputs

## 4.2 Definir el estimador (modelo supervisado)

En esta sección definimos un estimador de clasificación. Puedes cambiar el algoritmo
(por ejemplo, `LogisticRegression`, `RandomForestClassifier`, `GBTClassifier`) según lo que
quieras experimentar. En el ejemplo usaremos un **RandomForestClassifier**.

In [ ]:
# 4.3 Definir el estimador de clasificación
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    labelCol='label',
    featuresCol='features',
    seed=42
)

rf

In [ ]:
# 4.4 Construir el Pipeline completo
from pyspark.ml import Pipeline

stages = [label_indexer] + indexers + [encoder, assembler, rf]
pipeline = Pipeline(stages=stages)

pipeline

## 5. División de datos, validación cruzada y tuning

En esta sección:

- Dividiremos los datos en entrenamiento y prueba.
- Definiremos una malla de hiperparámetros con `ParamGridBuilder`.
- Configuraremos un `CrossValidator` para seleccionar el mejor modelo.
- Entrenaremos el pipeline y evaluaremos el resultado en el conjunto de prueba.

In [ ]:
# 5.1 Dividir datos en entrenamiento y prueba
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

print(f'Train: {train_df.count()} filas, Test: {test_df.count()} filas')

In [ ]:
# 5.2 Definir la malla de hiperparámetros y el CrossValidator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='rawPrediction', metricName='areaUnderROC')

param_grid = (ParamGridBuilder()
    .addGrid(rf.numTrees, [50, 100])
    .addGrid(rf.maxDepth, [5, 10])
    .build()
)

cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    parallelism=2
)

cv

In [ ]:
# 5.3 Entrenar el modelo con validación cruzada
cv_model = cv.fit(train_df)

best_model = cv_model.bestModel  # PipelineModel
best_rf = best_model.stages[-1]  # Última etapa: RandomForestClassificationModel

print(best_rf)

In [ ]:
# 5.4 Evaluar el mejor modelo en el conjunto de prueba
pred_test = cv_model.transform(test_df)

auc = evaluator.evaluate(pred_test)
print(f'ROC AUC en test: {auc:.4f}')

pred_test.select('Survived', 'prediction', 'probability').show(10, truncate=False)

## 6. Conclusiones

En esta sección, redacte sus **conclusiones** respecto al modelo entrenado:

- ¿Qué tan bien está rindiendo el modelo según las métricas obtenidas (por ejemplo, ROC AUC)?
- ¿Qué posibles mejoras podría aplicar (más features, otros modelos, mejor manejo de nulos, etc.)?
- ¿Qué limitaciones observa en el análisis realizado?

Escriba sus conclusiones en esta celda en formato markdown.